In [2]:
import pandas as pd
df=pd.read_csv('/Users/nat/Desktop/datasets/clickbait_data.csv')
df

,headline,clickbait
0,Should I Get Bings,1
1,Which TV Female Friend Group Do You Belong In,1
2,"The New ""Star Wars: The Force Awakens"" Trailer...",1
3,"This Vine Of New York On ""Celebrity Big Brothe...",1
4,A Couple Did A Stunning Photo Shoot With Their...,1
...,...,...
31995,"To Make Female Hearts Flutter in Iraq, Throw a...",0
31996,"British Liberal Democrat Patsy Calton, 56, die...",0
31997,Drone smartphone app to help heart attack vict...,0
31998,"Netanyahu Urges Pope Benedict, in Israel, to D...",0


In [3]:
df.isnull().sum()


headline     0
clickbait    0
dtype: int64

In [4]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/nat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
import string
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

def RemoveSpecialCharacters(sentence):
    return re.sub('[^a-zA-Z]+',' ',sentence)

def ConvertAndRemove(sentence):
    sentence = str(sentence)
    sentence = RemoveSpecialCharacters(sentence)
    # convert to lower case
    sentence = ConvertToLowerCase(sentence)
    return sentence

def ConvertToLowerCase(sentence):
    return sentence.lower()

def CleanText(sentence):
    sentence = str(sentence)

    # Remove stopwords
    STOPWORDS = stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure']
    # Remove punctuation
    nopunc = [char for char in sentence if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    sentence = ' '.join([word for word in nopunc.split() if word.lower() not in STOPWORDS])
    sentence = ConvertAndRemove(sentence)
    return sentence

In [6]:
print(CleanText('i am zyzzingrizzington bear i am heem what ok \asfa[] lm'))

zyzzingrizzington bear heem ok sfa lm


In [7]:
df['Text_cleaning'] = df.headline.apply(CleanText)
df.head()
# Cleaning dataset done

,headline,clickbait,Text_cleaning
0,Should I Get Bings,1,get bings
1,Which TV Female Friend Group Do You Belong In,1,tv female friend group belong
2,"The New ""Star Wars: The Force Awakens"" Trailer...",1,new star wars force awakens trailer give chills
3,"This Vine Of New York On ""Celebrity Big Brothe...",1,vine new york celebrity big brother fucking pe...
4,A Couple Did A Stunning Photo Shoot With Their...,1,couple stunning photo shoot baby learning inop...


In [9]:
# need to split datasets
X = df.Text_cleaning
y = df.clickbait
print('original length training:',len(X))
print('original length test:',len(y))

original length training: 32000
original length test: 32000


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print('new len x_train:',len(X_train))
print('new len x_test:',len(X_test))
print('new len y_train:',len(y_train))
print('new len y_test:',len(y_test))

new len x_train: 25600
new len x_test: 6400
new len y_train: 25600
new len y_test: 6400


In [11]:
#vectors
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

X_train_dtm

<25600x21436 sparse matrix of type '<class 'numpy.int64'>'
	with 152145 stored elements in Compressed Sparse Row format>

In [14]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()
X_train_clean = tfidf_transformer.fit_transform(X_train_dtm)

print(X_train_clean)


  (0, 20691)	0.27874184201961916
  (0, 17768)	0.3345529176182634
  (0, 15408)	0.443280710732536
  (0, 15002)	0.37952311039285885
  (0, 13192)	0.443280710732536
  (0, 12018)	0.32318125012329857
  (0, 5712)	0.30945596770049805
  (0, 3477)	0.2704345505044664
  (1, 19768)	0.4943954061330551
  (1, 19352)	0.6624730736498667
  (1, 8360)	0.5627633686403459
  (2, 19358)	0.398093544057493
  (2, 14980)	0.3044588790043617
  (2, 14484)	0.25078631831387677
  (2, 13284)	0.33813632294469903
  (2, 12879)	0.18494510864592098
  (2, 12716)	0.45227553340304516
  (2, 9559)	0.2966918040963563
  (2, 7673)	0.398093544057493
  (2, 7100)	0.2938265186498573
  (3, 12037)	0.31565021664326437
  (3, 10624)	0.41919574997204595
  (3, 10118)	0.402951847353606
  (3, 8075)	0.2854420765479582
  (3, 7531)	0.3409694430081179
  :	:
  (25596, 11813)	0.5860933147732315
  (25596, 9071)	0.3572209408805257
  (25596, 7075)	0.29993546844497326
  (25596, 6255)	0.43653766312518216
  (25597, 15954)	0.36744333021546094
  (25597, 15204)	

In [18]:
# test different models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn import metrics

In [19]:
Classifiers = [
{
   'label': 'Logistic Regression',
   'model': LogisticRegression(C=0.00000001,solver='liblinear',max_iter=200, multi_class='auto'),
},
{
    'label': 'SGD Classifier',
    'model': SGDClassifier(loss='log', warm_start=True, max_iter=1000, l1_ratio=0.03, penalty='l2', alpha=1e-4, fit_intercept=False),
},
{
    'label': 'KNeighbours',
    'model': KNeighborsClassifier(n_neighbors=15),
},
{
    'label': 'Decision Tree',
    'model': DecisionTreeClassifier(max_depth=10,random_state=101,max_features= None,min_samples_leaf=15),
},
{
   'label': 'Random Forest',
   'model': RandomForestClassifier(n_estimators=70, oob_score=True, n_jobs=-1,random_state=101,max_features= None,min_samples_leaf = 30),
}
]


In [20]:
Accuracy=[]
Model=[]
prediction = []
for c in Classifiers:
    try:
        classifier = c['model']
        fit = classifier.fit(X_train_clean, y_train)
        pred = fit.predict(X_test_dtm)
    except Exception:
        fit = classifier.fit(X_train_clean, y_train)
        pred = fit.predict(X_test_dtm)
    prediction.append(pred)
    accuracy = accuracy_score(pred,y_test)
    Accuracy.append(accuracy)
    Model.append(c['label'])
    print('Accuracy of '+c['label']+' is '+str(accuracy))

Accuracy of Logistic Regression is 0.68390625
Accuracy of SGD Classifier is 0.9421875


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


Accuracy of KNeighbours is 0.8678125
Accuracy of Decision Tree is 0.6415625
Accuracy of Random Forest is 0.8153125


In [21]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

pipe = Pipeline([('bow', CountVectorizer()), 
                 ('tfid', TfidfTransformer()),  
                 ('model', Classifiers[1]['model'])])
pipe.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


Pipeline(steps=[('bow', CountVectorizer()), ('tfid', TfidfTransformer()),
                ('model',
                 SGDClassifier(fit_intercept=False, l1_ratio=0.03, loss='log',
                               warm_start=True))])

In [30]:
y_pred = pipe.predict(X_test)
#sample clickbait from the dataset
test_sample1=['The New Star Wars: The Force Awakens Trailer Is Here To Give You Chills'] 

#sample not a clickbait from the dataset
test_sample2 = ['Scientology defector arrested after attempting to leave organization']  

#made up sample, expected to be clickbait
test_sample3 =['The Ultimate Guide to Making Money Online']

prediction = pipe.predict(test_sample3)
if prediction == 0:
  result = 'Not Clickbait'
else:
    result = 'Clickbait'
print(f'The headline is {result}')

The headline is Clickbait
